In [109]:
import pandas as pd
import geopandas as gpd
import os
import json
import colorcet as cc
colour_palette = cc.CET_R3

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook, reset_output

output_notebook()

### making data a bit easier to see
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

### should be outside the repo'
data_location = r'/Users/jj/code/strava_data/'

Loading BokehJS ...

In [125]:
raw = pd.read_csv(data_location+'all_activities_cleaned.csv', index_col=0, parse_dates=True)
raw = raw.apply(lambda x: pd.to_datetime(x) if 'date' in x.name else x)
raw

,name,distance,moving_time_seconds,elapsed_time_seconds,total_elevation_gain,type,sport_type,workout_type,id,start_date_local,achievement_count,kudos_count,comment_count,athlete_count,photo_count,trainer,commute,manual,private,visibility,flagged,gear_id,start_latlng,end_latlng,average_speed_kph,max_speed_kph,average_temp,average_watts,kilojoules,device_watts,has_heartrate,average_heartrate,max_heartrate,heartrate_opt_out,display_hide_heartrate_option,elev_high,elev_low,pr_count,total_photo_count,has_kudoed,suffer_score,average_cadence,max_watts,weighted_average_watts,athlete_id,athlete_resource_state,map_summary_polyline,bike_id,bike_primary,bike_name,bike_nickname,bike_retired,year,month,moving_time_minutes,moving_time_hours,elapsed_time_minutes,elapsed_time_hours,calories
0,Morning Mountain Bike Ride,24.9629,9598,13548,930.4,Ride,MountainBikeRide,10.0,8744397493,2023-03-20 09:31:46,33,1,0,1,0,False,False,False,False,everyone,False,b11952969,"[-41.29796063527465, 174.72158012911677]","[-41.297213891521096, 174.7220081090927]",9.3636,35.0208,24.0,130.6,1253.9,False,True,144.1,178.0,False,True,404.5,126.1,19,0,False,123.0,NaN,NaN,NaN,23883723,1,h_a{F{hli`@rE`Bx@jBjAZ`CuEdF|ATlAm@xD`@}DaA`DD...,b11952969,False,Canyon Spectral,Canyon Spectral,False,2023,2023-03,159.966667,2.666111,225.800000,3.763333,300.0
1,Afternoon Mountain Bike Ride,11.4628,5136,7891,389.0,Ride,MountainBikeRide,10.0,8744607827,2023-03-18 14:43:29,0,0,0,2,0,False,False,False,False,everyone,False,b11952969,"[-41.29956400953233, 174.78885202668607]","[-41.29894039593637, 174.78847458958626]",8.0352,33.9912,26.0,117.4,602.9,False,False,NaN,NaN,False,False,169.8,14.6,0,0,False,NaN,NaN,NaN,NaN,23883723,1,hia{Fimyi`@SMBHm@Qs@AQSFNCBIIBMGWWOCiAm@[F?DET...,b11952969,False,Canyon Spectral,Canyon Spectral,False,2023,2023-03,85.600000,1.426667,131.516667,2.191944,144.0
2,Lunch Ride,2.0372,278,775,0.0,Ride,Ride,NaN,8726797885,2023-03-17 11:49:16,0,0,0,1,0,False,False,False,True,only_me,False,b4083088,"[-41.28829252906144, 174.7669970523566]","[-41.2935836892575, 174.77822947315872]",26.3808,56.9376,27.0,26.4,7.3,False,True,113.5,136.0,False,True,111.6,-14.0,0,0,False,0.0,NaN,NaN,NaN,23883723,1,zj_{Fggui`@DK@GS]KIyCiAeAe@}@QkA]kCeAc@UKOBg@D...,b4083088,True,Avanti Giro 2,Avanti Giro 2,False,2023,2023-03,4.633333,0.077222,12.916667,0.215278,2.0
3,Lunch Ride,2.1672,727,773,132.0,Ride,Ride,NaN,8721532095,2023-03-16 12:29:03,0,0,0,1,0,False,False,False,True,only_me,False,b4083088,"[-41.293124444782734, 174.77520930580795]","[-41.28730338066816, 174.76581109687686]",10.7316,19.8360,28.0,146.0,106.1,False,True,125.4,145.0,False,True,134.4,32.6,0,0,False,3.0,NaN,NaN,NaN,23883723,1,`a`{F_xvi`@?NQv@?d@I^Wn@]lAi@xBGb@i@xAKl@M`A[x...,b4083088,True,Avanti Giro 2,Avanti Giro 2,False,2023,2023-03,12.116667,0.201944,12.883333,0.214722,25.0
4,Lunch Ride,2.1091,267,302,3.0,Ride,Ride,NaN,8721256990,2023-03-16 11:56:29,0,0,0,1,0,False,False,False,True,only_me,False,b4083088,"[-41.28733104094863, 174.76634301245213]","[-41.29523408599198, 174.77632116526365]",28.4364,52.9488,28.0,33.5,8.9,False,True,105.8,121.0,False,True,114.0,2.0,0,0,False,0.0,NaN,NaN,NaN,23883723,1,nj_{Fggui`@BGIUOQkB_AqAs@_HeCq@]ES?QFg@JIf@ErA...,b4083088,True,Avanti Giro 2,Avanti Giro 2,False,2023,2023-03,4.450000,0.074167,5.033333,0.083889,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,Radio dome ridge,19.8536,4766,4928,707.1,Ride,Ride,10.0,1121238293,2017-08-07 15:21:49,9,0,0,1,0,False,False,False,False,everyone,False,b4083088,"[-41.294885, 174.781158]","[-41.294907, 174.781201]",14.9976,71.2800,NaN,161.6,770.2,False,False,NaN,NaN,False,False,483.2,12.0,1,1,False,NaN,NaN,NaN,NaN,23883723,1,j{`{F_pwi`@|C~A`ADh@n@UlDeBdIdAp@Hf@cEnQUzB}@d...,b4083088,True,Avanti Giro 2,Avanti Giro 2,False,2017,2017-08,79.433333,1.323889,82.

In [235]:
### will return a list of colours of n length from the colour_palette
def get_colours(n):
    distance = int(len(colour_palette) / n)
    return [x for i, x in enumerate(colour_palette) if i % distance == 0]

### inputs (apart from df & order_by) must be lists - even if there is only one element
### will return a df whith cats and aggregated metrics
def aggregate(df, cat_columns, met_column=None, aggregation=None, order_by=None):

    raw_data = df[cat_columns]
    if type(cat_columns) == str:
        if not met_column and not aggregation:
            grouped = raw_data.value_counts().reset_index()
            grouped = grouped.rename(columns={'index':cat_columns, cat_columns:'count'})

        elif met_column and aggregation:
            raw_data = pd.concat([raw_data, df[met_column]], axis=1)
            grouped = raw_data.groupby(cat_columns).agg(aggregation).reset_index()
        
        if order_by: grouped = grouped.sort_values(order_by, ascending=False)
        grouped = grouped.to_dict('list')

    elif type(cat_columns) == list:
        if not met_column and not aggregation:
            grouped = raw_data.value_counts().unstack(fill_value=0)

        elif met_column and aggregation:
            raw_data = pd.concat([raw_data, df[met_column]], axis=1)
            grouped = raw_data.groupby(cat_columns).agg(aggregation).unstack(fill_value=0)
            grouped.columns = grouped.columns.get_level_values(1)
        if order_by: grouped = grouped.sort_values(order_by, ascending=False)
        grouped = {
            **{grouped.index.name:grouped.index.to_list()}, 
            **{cat_columns[1]:grouped.to_dict('list')}
        }

    else:
        return ('invalid_input')

    grouped['metadata'] = {'cat_columns':cat_columns, 'met_columns':met_column, 'aggregation':aggregation, 'order_by':order_by}

    return grouped

def custom_plot(x=None, title=None):
    p = figure(x_range=x, background_fill_color='whitesmoke', height=300, title=title)
    p.y_range.start = 0
    p.xgrid.grid_line_color=None
    p.xaxis.major_label_orientation = .8
    p.yaxis.minor_tick_line_color = None
    return p

###  case 1
# {x:[a, b, c], y:[count_a, count_b, count_c]}
# input = [a, a, b, c]
print('case1')
display(aggregate(raw, 'bike_name'))

### case 2
# {x:[a, b, c], y:[agg_a, agg_b, agg_c]}
# inputs = [a, a, b, c], metric, aggregation
print('case2')
display(aggregate(raw, 'bike_name', 'kudos_count', 'sum'))

### case 3
# {x:[a, b, c], z1: [count_a_z1, count_b_z1, count_c_z1], z2:[count_a_z2, count_b_z2, count_c_z2]}
# inputs = [[a, a, b, c], [z1, z2, z2, z3]]
print('case3')
display(aggregate(raw, ['bike_name', 'year']))

### case 4
# {x:[a, b, c], z1: [agg_a_z1, agg_b_z1, agg_c_z1], z2:[agg_a_z2, agg_b_z2, agg_c_z2]}
# inputs = [[a, a, b, c], [z1, z2, z2, z3]], metric_aggregation
print('case4')
display(aggregate(raw, ['bike_name', 'year'], 'kudos_count', 'sum'))



case1


{'bike_name': ['Avanti Giro 2',
  'Cervelo Aspero',
  'Transition Smuggler',
  'Canyon Spectral',
  'Fat Bike'],
 'count': [681, 203, 136, 19, 10],
 'metadata': {'cat_columns': 'bike_name',
  'met_columns': None,
  'aggregation': None,
  'order_by': None}}

case2


{'bike_name': ['Avanti Giro 2',
  'Canyon Spectral',
  'Cervelo Aspero',
  'Fat Bike',
  'Transition Smuggler'],
 'kudos_count': [457, 30, 284, 18, 366],
 'metadata': {'cat_columns': 'bike_name',
  'met_columns': 'kudos_count',
  'aggregation': 'sum',
  'order_by': None}}

case3


{'bike_name': ['Avanti Giro 2',
  'Canyon Spectral',
  'Cervelo Aspero',
  'Fat Bike',
  'Transition Smuggler'],
 'year': {2017: [56, 0, 0, 0, 0],
  2018: [75, 0, 0, 2, 0],
  2019: [164, 0, 0, 4, 0],
  2020: [172, 0, 44, 1, 0],
  2021: [31, 0, 114, 1, 63],
  2022: [156, 0, 27, 2, 71],
  2023: [27, 19, 18, 0, 2]},
 'metadata': {'cat_columns': ['bike_name', 'year'],
  'met_columns': None,
  'aggregation': None,
  'order_by': None}}

case4


{'bike_name': ['Avanti Giro 2',
  'Canyon Spectral',
  'Cervelo Aspero',
  'Fat Bike',
  'Transition Smuggler'],
 'year': {2017: [22, 0, 0, 0, 0],
  2018: [48, 0, 0, 1, 0],
  2019: [121, 0, 0, 7, 0],
  2020: [218, 0, 51, 1, 0],
  2021: [19, 0, 168, 4, 185],
  2022: [20, 0, 41, 5, 172],
  2023: [9, 30, 24, 0, 9]},
 'metadata': {'cat_columns': ['bike_name', 'year'],
  'met_columns': 'kudos_count',
  'aggregation': 'sum',
  'order_by': None}}

In [258]:
def bar_chart(x, y, t=None):
    c = get_colours(len(x)-1)
    p = custom_plot(x=x, title=t)
    p.vbar(x=x, top=y, width=.9, fill_color=c, line_color='white', line_width=3, line_join='round')
    show(p)

data = aggregate(raw, cat_columns='type')
title = 'Count of Activities by Type'
bar_chart(data['type'], data['count'], title)


In [259]:
data = aggregate(raw, cat_columns='bike_name', met_column='distance', aggregation='sum', order_by='distance')
title = 'Distance by Bike'
bar_chart(data['bike_name'], data['distance'], title)

In [260]:
def stacked_bar_chart(x, ys, t):

    c = get_colours(len(ys.keys()))
    p = custom_plot(x=x, title=t)

    ### iterate through second cat column
    bottom = [0 for z in x]
    for i, (k, v) in enumerate(ys.items()):
        top = [x+y for x, y in zip(bottom, v)]
        p.vbar(x=x, bottom=bottom, top=top, width=.9, color=c[i], line_color='white', line_width=1, line_join='round', legend_label=str(k))
        bottom = top

    p.legend.location = 'top_left'

    show(p)

data = aggregate(raw, cat_columns=['year', 'bike_name'])
title = 'Activities by Bike and Year'
stacked_bar_chart([str(z) for z in data['year']], data['bike_name'], t=title)

In [265]:
data = aggregate(raw, ['month', 'bike_name'], 'distance', 'sum')
title = 'Distance by Bike and Month'
stacked_bar_chart(data['month'], data['bike_name'], title)